In [1]:
import pandas as pd
import numpy as np
from statistics import *

In [2]:
temperature_original = pd.read_csv("4.csv")

In [3]:
temperature_original1 = temperature_original
temperature_original=temperature_original.drop(['Date'],axis=1)
temperature_original.describe()

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,...,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax,Next_Tmin
count,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,...,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000
mean,13.000000,29.680232,23.196606,56.403017,87.715411,29.520808,23.479000,7.221373,62.084028,0.558724,...,0.671193,0.467084,0.458269,37.544720,126.991400,61.854944,1.256692,5341.549330,30.239639,22.922619
std,7.211568,3.094748,2.417668,15.022533,9.793997,3.075903,2.356118,2.507929,33.773612,1.939613,...,2.582990,2.243802,2.266888,0.050335,0.079406,54.256953,1.369833,428.996445,3.176239,2.484868
min,1.000000,20.000000,11.300000,19.794666,20.000000,17.624954,14.272646,2.882580,-13.603212,0.000000,...,0.000000,0.000000,0.000000,37.456200,126.826000,12.370000,0.098500,4329.520508,17.400000,11.300000
25%,7.000000,27.700000,21.600000,45.735747,84.040119,27.601625,22.032778,5.686498,36.778658,0.148339,...,0.000000,0.000000,0.000000,37.510200,126.937000,28.700000,0.271300,4999.018555,28.200000,21.300000
50%,13.000000,29.900000,23.400000,54.828714,89.699505,29.662273,23.735942,6.563068,56.487289,0.320110,...,0.000000,0.000000,0.000000,37.550700,126.995000,45.716000,0.618000,5436.345215,30.500000,23.100000
75%,19.000000,32.000000,24.875000,67.032715,93.703491,31.682567,25.133460,8.092060,83.891182,0.584296,...,0.024224,0.009685,0.000686,37.577600,127.042000,59.832400,1.767800,5728.316406,32.600000,24.600000
max,25.000000,37.600000,29.900000,98.524734,100.000153,38.542255,29.619342,21.857621,213.414006,20.000000,...,21.621661,20.000000,20.000000,37.645000,127.135000,212.335000,5.178200,5992.895996,38.900000,29.800000


In [4]:
#temperature_original = temperature_original.drop(['Date'],axis=1)
#temperature_original.info()
#temp_train_features = temperature_original.drop(['Next_Tmax','Next_Tmin'],axis=1)
#temp_train_labels = temperature_original[['Next_Tmax','Next_Tmin']].copy()

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [6]:
my_pipeline  =Pipeline ([
    
    ("imputer", SimpleImputer(strategy="median")),
    ("std_scale",StandardScaler())
])

In [7]:
temp_transform = my_pipeline.fit_transform(temperature_original)
#temp_label_transform = my_pipeline.fit_transform(temp_train_labels)
#temp_train_features.shape

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit
split1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split1.split(temperature_original, temperature_original['station']):
        strat_train_set = temperature_original.loc[train_index]
        strat_test_set = temperature_original.loc[test_index]

In [9]:
temp_train_features = strat_train_set.drop(['Next_Tmax','Next_Tmin'],axis=1)
temp_train_labels = strat_train_set[['Next_Tmax']].copy()

In [10]:
#strat_train_set.info()

In [11]:
#from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [12]:
#model = LinearRegression()
model = RandomForestRegressor()
temp_train_labels['Next_Tmax'].fillna(median)
temp_transform1 = my_pipeline.fit_transform(temp_train_features)

In [17]:
model.fit(temp_transform1,temp_train_labels)

<ipython-input-17-03baae101eca>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(temp_transform1,temp_train_labels)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
temp_transform1

array([[-0.83205029, -0.1626324 , -0.48568894, ..., -0.91210682,
        -0.8455532 ,  0.91648482],
       [ 1.38675049,  1.16559752,  2.34368263, ..., -0.8528    ,
        -0.80401261, -0.19323674],
       [-0.5547002 , -0.35700751, -0.03462971, ..., -0.20134652,
        -0.6163134 ,  0.85259402],
       ...,
       [ 1.66410059, -0.94013285,  0.82648338, ..., -0.77913095,
        -0.71939831,  1.01387147],
       [ 0.69337525, -0.03304899,  0.78547799, ..., -0.09971981,
        -0.01853776, -1.55808111],
       [ 0.97072534,  0.61486804,  0.66246184, ...,  1.56117357,
         2.53544186,  0.5350219 ]])

In [20]:
temp_predict = model.predict(temp_transform1)

In [21]:
temp_predict

array([29.396, 35.101, 30.57 , ..., 26.226, 32.117, 29.281])

In [22]:
  temp_transform1.shape

(6200, 22)

In [23]:
mse = mean_squared_error(temp_predict,temp_train_labels)

In [24]:
mse

0.18875365354838716

In [25]:
rmse = np.sqrt(mse)
rmse

0.4344578846659215

In [26]:
0

0

In [35]:
some_data = temp_train_features.iloc[:2]

In [36]:
some_labels = temp_train_labels.iloc[:2]

In [37]:
x = model.predict(some_data)
x

array([34.776, 34.776])

In [38]:
some_labels

,Next_Tmax
4956,29.8
7147,35.3


In [39]:
mse1 = mean_squared_error(x,some_labels)

In [40]:
mse1

12.517576000000105

In [41]:
rmse = np.sqrt(mse1)

In [42]:
rmse

3.538018654557958